In [102]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import pyfolio as pf

# Define the start and end dates for data collection
start_date = dt.datetime.now() - dt.timedelta(days=3*365)  # 3 years ago
end_date = dt.datetime.now()

# Get a list of F&O listed stocks in India
fo_stocks = ['TCS.NS', 'RELIANCE.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS', 'TATAMOTORS.NS', 'PVR.NS', 'ACC.NS']  # Example stocks, there are total 198 stocks
l = len(fo_stocks)

# Iterate over each stock
for stock in fo_stocks:
    # Get the historical OHLC data using yfinance
    data = yf.download(stock, start=start_date, end=end_date)

data.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-07-01,1339.949951,1340.000000,1308.650024,1325.949951,1280.495728,1411786
2020-07-02,1329.800049,1340.000000,1318.250000,1328.849976,1283.296509,1008546
2020-07-03,1330.000000,1337.550049,1306.000000,1314.949951,1269.872925,1019896
2020-07-06,1317.000000,1334.949951,1307.000000,1328.000000,1282.475586,1070274
2020-07-07,1327.900024,1327.900024,1307.849976,1313.099976,1268.086304,796319


In [115]:
def calculate_rolling_returns(data, window=52):
    returns = []
    for i in range(window, len(data)):
        start_value = data[i - window]
        end_value = data[window]
        total_return = (end_value - start_value) / start_value
        returns.append(total_return)
    return pd.Series(returns)

In [116]:
Rolling_returns = calculate_rolling_returns(data['Adj Close'], 52)

Rolling_returns


0     -0.003318
1     -0.005494
2      0.005019
3     -0.004857
4      0.006435
         ...   
689   -0.254529
690   -0.266925
691   -0.265427
692   -0.274035
693   -0.277957
Length: 694, dtype: float64

In [117]:
# Calculate the 52-week rolling returns
data['RollingReturns'] = Rolling_returns

# Sort the stocks based on rolling returns and select the top 5 performers
top_performers = data.sort_values('RollingReturns', ascending=False).head(l)

# top_performers = data.groupby('Symbol').tail(1).nlargest(5, 'RollingReturns')

top_performers


,Open,High,Low,Close,Adj Close,Volume,Price,RollingReturns
Date,,,,,,,,
2020-07-01,1339.949951,1340.000000,1308.650024,1325.949951,1280.495728,1411786,1324.325012,NaN
2020-07-02,1329.800049,1340.000000,1318.250000,1328.849976,1283.296509,1008546,1329.125000,NaN
2020-07-03,1330.000000,1337.550049,1306.000000,1314.949951,1269.872925,1019896,1321.775024,NaN
2020-07-06,1317.000000,1334.949951,1307.000000,1328.000000,1282.475586,1070274,1320.974976,NaN
2020-07-07,1327.900024,1327.900024,1307.849976,1313.099976,1268.086304,796319,1317.875000,NaN
2020-07-08,1313.099976,1320.699951,1290.599976,1312.900024,1267.893188,1102284,1305.649963,NaN
2020-07-09,1319.000000,1333.750000,1306.300049,1322.900024,1277.550415,1366285,1320.025024,NaN
2020-07-10,1321.500000,1328.000000,1306.099976,1310.250000,1265.333984,832336,1317.049988,NaN


In [118]:
# Define the entry price as the close price of the most recent data point
entry_price = top_performers.iloc[-1]['Close']

# Define the stop loss as 2 times the Average True Range (ATR) on the entry price
atr = top_performers['High'] - top_performers['Low']
stop_loss = entry_price - (2 * atr)

stop_loss

Date
2020-07-01    1247.550049
2020-07-02    1266.750000
2020-07-03    1247.149902
2020-07-06    1254.350098
2020-07-07    1270.149902
2020-07-08    1250.050049
2020-07-09    1255.350098
2020-07-10    1266.449951
dtype: float64

In [119]:
# Initialize an empty DataFrame to store the strategy performance
strategy_returns = pd.DataFrame()

# Calculate the weekly returns based on rebalancing the portfolio
weekly_returns = top_performers['Close'].pct_change()

# Combine the stock returns with the strategy returns DataFrame
strategy_returns = pd.concat([strategy_returns, weekly_returns], axis=1)

strategy_returns

,Close
2020-07-01 00:00:00,NaN
2020-07-02 00:00:00,0.002187
2020-07-03 00:00:00,-0.010460
2020-07-06 00:00:00,0.009924
2020-07-07 00:00:00,-0.011220
2020-07-08 00:00:00,-0.000152
2020-07-09 00:00:00,0.007617
2020-07-10 00:00:00,-0.009562


In [120]:
# Rename the columns of the strategy returns DataFrame with stock symbols
# strategy_returns.set_axis(fo_stocks, axis = 0, inplace = True)

strategy_returns['Stocks'] = fo_stocks

strategy_returns

,Close,Stocks
2020-07-01 00:00:00,NaN,TCS.NS
2020-07-02 00:00:00,0.002187,RELIANCE.NS
2020-07-03 00:00:00,-0.010460,HDFCBANK.NS
2020-07-06 00:00:00,0.009924,INFY.NS
2020-07-07 00:00:00,-0.011220,ICICIBANK.NS
2020-07-08 00:00:00,-0.000152,TATAMOTORS.NS
2020-07-09 00:00:00,0.007617,PVR.NS
2020-07-10 00:00:00,-0.009562,ACC.NS


In [121]:
# Perform the backtest and analyze the strategy using pyfolio
returns = strategy_returns.dropna()

returns

,Close,Stocks
2020-07-02 00:00:00,0.002187,RELIANCE.NS
2020-07-03 00:00:00,-0.010460,HDFCBANK.NS
2020-07-06 00:00:00,0.009924,INFY.NS
2020-07-07 00:00:00,-0.011220,ICICIBANK.NS
2020-07-08 00:00:00,-0.000152,TATAMOTORS.NS
2020-07-09 00:00:00,0.007617,PVR.NS
2020-07-10 00:00:00,-0.009562,ACC.NS


In [122]:
pf.create_full_tear_sheet(returns['Close'])

c:\Users\annar\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
c:\Users\annar\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyfolio\plotting.py:648: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for stat, value in perf_stats[column].iteritems():


Start date,2020-07-02
End date,2020-07-10
Total months,0
,Backtest
Annual return,-34.9%
Cumulative returns,-1.2%
Annual volatility,14.0%
Sharpe ratio,-2.99
Calmar ratio,-24.91
Stability,0.30
Max drawdown,-1.4%


IndexError: index -1 is out of bounds for axis 0 with size 0